In [ ]:
import pandas as pd

**Cleaning 1: spotify-2023.csv - Converting and Sorting Columns.**

1. The Spotify-2023.csv file is cleaned below by reading only the columns needed. The 'track_name' and 'artist(s)_name' columns identify the song and artist, the 'streams' column identifies the popularity of the song, and the 'bpm', 'key', and 'mode' columns identify the song's musical attributes.

2. Convert the 'streams' column to numeric using to_numeric() forcing errors to NaN 

3. Drop rows with NaN using .dropna()

4. Convert 'streams' column to integers using .astype(int)

5. Sort the DataFrame by 'streams' column in descending order using .sort() to have the most popular songs at the top of the DataFrame.

In [ ]:
columns_to_read = ['track_name', 'artist(s)_name', 'streams', 'bpm', 'key', 'mode']
df_2023 = pd.read_csv('spotify-2023.csv', usecols= columns_to_read, encoding= 'latin-1')

# Convert 'streams' column to numeric, forcing errors to NaN
df_2023['streams'] = pd.to_numeric(df_2023['streams'], errors='coerce')

# Drop rows with NaN if necessary
df_2023 = df_2023.dropna()

# Convert 'streams' column to integers
df_2023['streams'] = df_2023['streams'].astype(int)

# Sort the DataFrame by 'streams' column in descending order
df_2023_desc = df.sort_values(by='streams', ascending=False)

print(df_2023_desc)

**Interpretation/Analysis 1: Most Common Attributes.**

Rather than tracking the songs simply based on popularity alone, we will also take a look at each song's musical attributes.

After having the initial DataFrame cleaned, we want to find the most common musical attributes for the entire DataFrame. 

We use .mode() to find the most common bpm, key, and mode used for each song. They are:

BPM= 120 KEY= C# MODE= Major


In [ ]:
common_bpm = df_2023_desc['bpm'].mode()[0]
common_key = df_2023_desc['key'].mode()[0]
common_mode = df_2023_desc['mode'].mode()[0]

print(common_bpm, 'is the most common BPM in the entire DataFrame.')
print(common_key, 'is the most common KEY in the entire DataFrame.')
print(common_mode, 'is the most common MODE in the entire DataFrame.')

**Interpretation/Analysis 2: Statistics of Attributes.**

Now, we will calculate *how often* the the most common attributes occur in the entire DataFrame. This will take 4 steps.

1. Count how many times an attribute occures in the column using .value_counts().

2. Count the total number of entries for the column using .count().

3. Divide the attribute count and total count, then multiply by 100 to get the percentage.

4. Round the percentage to the nearest 2 decimal points using the round() function.

In [ ]:
#BPM Statistics
count_bpm = df_2023_desc['bpm'].value_counts()[common_bpm]
total_bpm = df_2023_desc['bpm'].count()
percentage_bpm = (count_bpm / total_bpm) * 100
percentage_bpm = round(percentage_bpm, 2)

#KEY Statistics
count_key = df_2023_desc['key'].value_counts()[common_key]
total_key = df_2023_desc['key'].count()
percentage_key = (count_key / total_key) * 100
percentage_key = round(percentage_key, 2)

#MODE Statistics
count_mode = df_2023_desc['mode'].value_counts()[common_mode]
total_mode = df_2023_desc['mode'].count()
percentage_mode = (count_mode / total_mode) * 100
percentage_mode = round(percentage_mode, 2)


print(common_bpm, 'is the most common BPM of all the songs in the 2023 DataFrame. It occurs', count_bpm, 'times.', 'This is', percentage_bpm, 'percent of the entire DataFrame.')
print(common_key, 'is the most common KEY of all the songs in the 2023 DataFrame. It occurs', count_key, 'times.', 'This is', percentage_key, 'percent of the entire DataFrame.')
print(common_mode, 'is the most common MODE of all the songs in the 2023 DataFrame. It occurs', count_mode, 'times.', 'This is', percentage_mode, 'percent of the entire DataFrame.')

**Now let's take a look at the top 10 songs of the DataFrame**

In [ ]:
df_top10 = df_2023_desc.head(10)

print(df_top10)

**Interpretation/Analysis 3: Statistics of Attributes in Top 10 songs.**

Now, we will calculate how often the the most common attributes occur in only the top 10 songs.

We quickly find that the calculations above in Interpretation/Analysis 3 might work, but can raise errors if those attributes don't appear in the DataFrame. We will add some code to prevent errors, specifically in the BPM statistics where the most common BPM does not appear in the top 10.

1. Count how many times an attribute occures in the column **using .get().** This will default to 0 if the value you are trying to find is not present (which is the case with BPM.)

2. Count the total number of entries for the column using .count().

3. Divide the attribute count and total count, then multiply by 100 to get the percentage.

4. Round the percentage to the nearest 2 decimal points using the round() function.



In [ ]:
#BPM Statistics in the Top 10 songs
count_bpm_top10 = df_top10['bpm'].value_counts().get(common_bpm, 0) #Get count, or default to 0 if necessary
total_bpm_top10 = df_top10['bpm'].count()
percentage_bpm_top10 = (count_bpm_top10 / total_bpm_top10) * 100
percentage_bpm_top10 = round(percentage_bpm_top10, 2)

#Key Statistics in the Top 10 songs
count_key_top10 = df_top10['key'].value_counts().get(common_key, 0) #Get count, or default to 0 if necessary
total_key_top10 = df_top10['key'].count()
percentage_key_top10 = (count_key_top10 / total_key_top10) * 100
percentage_key_top10 = round(percentage_key_top10, 2)

#Mode Statistics in the Top 10 songs
count_mode_top10 = df_top10['mode'].value_counts().get(common_mode, 0) #Get count, or default to 0 if necessary
total_mode_top10 = df_top10['mode'].count()
percentage_mode_top10 = (count_mode_top10 / total_mode_top10) * 100
percentage_mode_top10 = round(percentage_mode_top10, 2)

print(common_bpm, 'BPM occurs', count_bpm_top10, 'times in the top-ten 2023 songs. This is', percentage_bpm_top10, 'percent of the top-ten songs, as opposed to', percentage_bpm, 'percent of the entire DataFrame.')
print(common_key, 'occurs', count_key_top10, 'times in the top-ten 2023 songs. This is', percentage_key_top10, 'percent of the top-ten songs, as opposed to', percentage_key, 'percent of the entire DataFrame.')
print(common_mode, 'occurs', count_mode_top10, 'times in the top-ten 2023 songs. This is', percentage_mode_top10, 'percent of the top-ten songs, as opposed to', percentage_mode, 'percent of the entire DataFrame.')


**Cleaning 2: Spotify-2024.csv - Converting and Sorting Columns.**

We have cleaned and analyzed the spotify-2023.csv file. The next part of this project will look at the top-ten 2023 songs, and compare their 'streams' count from 2023 to their 2024 current count. 

First, we will read, and clean, the spotify-2024.csv file. We can use mostly the same method as we did for the 2023.csv file. However, we need to add two steps. The 2024.csv 'Spotify Streams' contains commas, which results in errors. The new steps to clean the file are as follows:

1. **NEW** Convert 'Spotify Streams' to a string using .astype(str)

2. **NEW** Check for commas, and replace if present using str.replace

3. Convert the 'streams' column to numeric using to_numeric() forcing errors to NaN 

4. Drop rows with NaN using .dropna()

5. Convert 'Spotify Streams' column to integers using .astype(int)

6. Sort the DataFrame by 'streams' column in descending order using .sort() to have the most popular songs at the top of the DataFrame.

In [ ]:

columns_to_read = ['Track', 'Artist', 'Spotify Streams']
df_2024 = pd.read_csv('spotify-2024.csv', usecols=columns_to_read, encoding='latin-1')


# NEW: Convert to string for inspection
df_2024['Spotify Streams'] = df_2024['Spotify Streams'].astype(str)

# NEW: Check for commas
df_2024['Spotify Streams'] = df_2024['Spotify Streams'].str.replace(',', '')  #<-- Remove commas if present

#Convert 'Spotify Streams' to numeric, forcing errors to NaN
df_2024['Spotify Streams'] = pd.to_numeric(df_2024['Spotify Streams'], errors='coerce')

# Drop rows with NaN values'
df_2024 = df_2024.dropna()

# Convert 'Spotify Streams' column to integers
df_2024['Spotify Streams'] = df_2024['Spotify Streams'].astype(int)

# Sort the DataFrame by 'Spotify Streams' in descending order
df_2024_desc = df_2024.sort_values(by='Spotify Streams', ascending=False)

print(df_2024_desc)